# Toronto Shelter Optimization

### Import Packages

In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import *

## Data used in model construction

In [2]:
#name list for current shelter
Name_current_shelter = ['s1', 's2', 's3', 's4', 's5' , 's6','s7','s8', 's9','s10','s11','s12','s13','s14','s15','s16','s17','s18','s19','s20','s21','s22','s23','s24','s25','s26','s27','s28','s29','s30',
                        's31','s32','s33','s34','s35','s36','s37','s38','s39','s40','s41','s42','s43','s44','s45','s46','s47','s48','s49','s50','s51',
                        's52','s53','s54','s55','s56','s57','s58','s59','s60','s61','s62','s63','s64','s65','s66','s67','s68','s69','s70','s71','s72','s73','s74','s75']
#capacity for current shelter
Cap_current_shelter = [210,152,82,42,76,56,560,888,94,28,40,90,88,51,28,120,50,40,32,252,48,110,50,25,110,25,110,34,21,74,48,45,40,10,60,5,28,26,37,5,70,25,90,30,75,90,83,45,40,71,20,106,90,94,19,108,290,60,45,89,72,348,397,305,40,46,22,40,44,30,27,89,68,100,320]

#categories of shelter based on size
size_of_shelter = ['small', 'medium', 'large']

#daily operating costs per homeless people
#each for the cost in small, medium and large shelters
daily_individual_cost = [85, 65, 50]

#construction cost for new shelter, each below for small, medium and large
construction_cost = [1400000, 2460000, 4900000]

#capacity for new shelters, each below for small, medium and large 
cap_new_shelters = [50,100,200] 

## Task: Model 1

In [3]:
model = gp.Model("Shelter Construction Optimization")

Academic license - for non-commercial use only - expires 2022-09-25
Using license file /Users/zorak/gurobi.lic


### Define decision variables

In [4]:
#Define the DVs
#x is the number of new shelters to be built in difference sizes
x = model.addVars(len(size_of_shelter), lb = 0, vtype = GRB.INTEGER, name = ["number_of_"+ i for i in size_of_shelter])
x

#y is the number of occupancy in each of the current shelters
y = model.addVars(len(Name_current_shelter), lb = 0, vtype = GRB.INTEGER, name = ['occupancy_' + i for i in Name_current_shelter])
y



{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>,
 5: <gurobi.Var *Awaiting Model Update*>,
 6: <gurobi.Var *Awaiting Model Update*>,
 7: <gurobi.Var *Awaiting Model Update*>,
 8: <gurobi.Var *Awaiting Model Update*>,
 9: <gurobi.Var *Awaiting Model Update*>,
 10: <gurobi.Var *Awaiting Model Update*>,
 11: <gurobi.Var *Awaiting Model Update*>,
 12: <gurobi.Var *Awaiting Model Update*>,
 13: <gurobi.Var *Awaiting Model Update*>,
 14: <gurobi.Var *Awaiting Model Update*>,
 15: <gurobi.Var *Awaiting Model Update*>,
 16: <gurobi.Var *Awaiting Model Update*>,
 17: <gurobi.Var *Awaiting Model Update*>,
 18: <gurobi.Var *Awaiting Model Update*>,
 19: <gurobi.Var *Awaiting Model Update*>,
 20: <gurobi.Var *Awaiting Model Update*>,
 21: <gurobi.Var *Awaiting Model Update*>,
 22: <gurobi.Var *Awaiting Model Update*>,
 23: <gurobi.Var *Awa

### Add constraints

In [5]:
#to add constraints
#for each of the current shelters, occupancy <= capacity
for i in range(len(Name_current_shelter)):
    model.addConstr(y[i] <= Cap_current_shelter[i], name="capacity_of_shelter"+ str(i+1))

#for total demand <= total capacity 
model.addConstr(sum(y[i] for i in range(len(Name_current_shelter))) + sum(x[i]*cap_new_shelters[i] for i in range(len(size_of_shelter))) >= 8700)


#for loop to calculate to the annual operating cost for 3 sizes
#individual operating cost for current shelters per day * 365 day * number of people in three sizes
a=[]
b=[]
c=[]
for i in range(len(Name_current_shelter)):
    if Cap_current_shelter[i] <= 50:       #individual cost for small shelters 
        a.append(y[i]*daily_individual_cost[0]*365)
    elif Cap_current_shelter[i] <= 100:    #individual cost for medium shelters 
        b.append(y[i]*daily_individual_cost[1]*365)
    else:                                 #individual cost for large shelters 
        c.append(y[i]*daily_individual_cost[2]*365)

        
#for total cost <= government budget (in a year)
#total cost measured by: individual operating cost for current shelters per day * 365 day * number of people + construction fee for new shelters + individual operating cost for new shelters per day * 365 day * number of people       
model.addConstr(sum(a)+sum(b)+sum(c) + sum(x[j]*construction_cost[j] + x[j]*daily_individual_cost[j]*365 for j in range(len(size_of_shelter))) <= 663200000)





<gurobi.Constr *Awaiting Model Update*>

### Set objective function

In [6]:
cost = sum(a)+sum(b)+sum(c) + sum(x[j]*construction_cost[j] + x[j]*daily_individual_cost[j]*365 for j in range(len(size_of_shelter)))
model.setObjective(cost, GRB.MINIMIZE)


In [7]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 77 rows, 78 columns and 231 nonzeros
Model fingerprint: 0x5cb6bcf3
Variable types: 0 continuous, 78 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+06]
  Objective range  [2e+04, 5e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 7e+08]
Found heuristic solution: objective 2.166298e+08
Presolve removed 75 rows and 72 columns
Presolve time: 0.00s
Presolved: 2 rows, 6 columns, 12 nonzeros
Variable types: 0 continuous, 6 integer (0 binary)

Root relaxation: objective 1.845615e+08, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.8456e+08    0    1 2.1663e+08 1.8456e+08  14.8%     -    0s
H    0     0                    1.846464e+08 1.8456e+08  0.05%     - 

In [8]:
for var in model.getVars():
    if var.x >= 0:
        print(var.varName, "=", round(var.x,0))
        

number_of_small = 0.0
number_of_medium = 1.0
number_of_large = 12.0
occupancy_s1 = 210.0
occupancy_s2 = 152.0
occupancy_s3 = 82.0
occupancy_s4 = -0.0
occupancy_s5 = 76.0
occupancy_s6 = 56.0
occupancy_s7 = 560.0
occupancy_s8 = 888.0
occupancy_s9 = 94.0
occupancy_s10 = -0.0
occupancy_s11 = -0.0
occupancy_s12 = 90.0
occupancy_s13 = 88.0
occupancy_s14 = 51.0
occupancy_s15 = -0.0
occupancy_s16 = 120.0
occupancy_s17 = -0.0
occupancy_s18 = -0.0
occupancy_s19 = -0.0
occupancy_s20 = 252.0
occupancy_s21 = -0.0
occupancy_s22 = 110.0
occupancy_s23 = -0.0
occupancy_s24 = -0.0
occupancy_s25 = 110.0
occupancy_s26 = -0.0
occupancy_s27 = 110.0
occupancy_s28 = -0.0
occupancy_s29 = -0.0
occupancy_s30 = 74.0
occupancy_s31 = -0.0
occupancy_s32 = -0.0
occupancy_s33 = -0.0
occupancy_s34 = -0.0
occupancy_s35 = 60.0
occupancy_s36 = -0.0
occupancy_s37 = 2.0
occupancy_s38 = -0.0
occupancy_s39 = -0.0
occupancy_s40 = -0.0
occupancy_s41 = 70.0
occupancy_s42 = -0.0
occupancy_s43 = 90.0
occupancy_s44 = -0.0
occupancy

### Solution from model1:

In this case, Gurobi suggests building 1
medium and 12 large shelters in order to accommodate the homeless people in Toronto. And the
minimized annual cost is $184,598,975. 

According to the results, optimal average daily occupancy
for about half of the existing shelters are zero, indicating that it is meaningless to keep those
shelters operating. In this case, the government can choose to shut down those shelters.

Although this model has good intentions, its optimal solutions do not seem to be reasonable, and
the nature of the solution is not compatible with our understanding of the problem. Building 13
new shelters in total while leaving half of the existing ones unused may be too much of a burden
for the City of Toronto. First of all, there will still be expenditures for maintenance even if the
shelters are left unused. Also, if the government chooses to shut down an existing shelter, it will
be subject to additional spendings. In reality, especially during a global pandemic, the
government will not be happy with such a solution that suggests closing half of the existing
shelters because it will cause huge wastes. Furthermore, the optimal solution provided by this model is not compatible with our initial intuitions. We want to leverage the existing shelters as
much as possible before building new ones so that we are not wasting the money from taxpayers
in Toronto. We understand the importance of leveraging the current resources. Considering the
time needed for construction, government approval, as well as recruitment, we realize it takes a
long time for new shelters to be put into use officially. In this rapidly changing world, we are not
sure whether the demand from homeless people can still be satisfied when new shelters are ready
years later. As a result, we need to make an improvement on this model.

## Task: Model2

### New assumption

Assume even for those shelters without being fully occupied, they are still paying for maintainence. And the maintainence fee is 50, 40, 30 per the empty spot for small, medium and large shelters respectively.

Methodology:
To include the extra cost into the model, we construct another for loop. We first
divide the existing shelters based on their capacity (small, medium, and large), and then calculate
the number of empty spots by using the corresponding capacity minus the occupancy in each of
the current shelters. Since the cost of each empty spot is measured on a daily basis, we multiply
it by 365 days as we did before to calculate the annual total cost. As such, for shelters of each
size, we multiply the number of empty spots with the annual cost of each unused spot to get the
annual operating cost for unused or seldomly-used shelters.

In [10]:
daily_empty_spot_cost = [50, 40, 30]

### Model 2 construction - decision variables & constraints

In [11]:
model2 = gp.Model("Shelter Construction Optimization_Extension")
#Define the DVs
#x is the number of new shelters to be built in difference sizes
x = model2.addVars(len(size_of_shelter), lb = 0, vtype = GRB.INTEGER, name = ["number_of_"+ i for i in size_of_shelter])
x

#y is the number of occupancy in each of the current shelters
y = model2.addVars(len(Name_current_shelter), lb = 0, vtype = GRB.INTEGER, name = ['occupancy_' + i for i in Name_current_shelter])
y

#to add constraints
#for each of the current shelters, occupancy <= capacity
for i in range(len(Name_current_shelter)):
    model2.addConstr(y[i] <= Cap_current_shelter[i], name="capacity_of_shelter"+ str(i+1))

#for total demand <= total capacity 
model2.addConstr(sum(y[i] for i in range(len(Name_current_shelter))) + sum(x[i]*cap_new_shelters[i] for i in range(len(size_of_shelter))) >= 8700)


#individual operating cost for current shelters per day * 365 day * number of people in three sizes
a=[]
b=[]
c=[]
for i in range(len(Name_current_shelter)):
    if Cap_current_shelter[i] <= 50:       #individual cost for small shelters
        a.append(y[i]*daily_individual_cost[0]*365)
    elif Cap_current_shelter[i] <= 100:    #individual cost for medium shelters
        b.append(y[i]*daily_individual_cost[1]*365)
    else:                                 #individual cost for large shelters
        c.append(y[i]*daily_individual_cost[2]*365)


#operating cost per empty spot * 365 day * number of empty spot in three sizes
d=[]
e=[]
f=[]
for i in range(len(Name_current_shelter)): 
    if Cap_current_shelter[i] <= 50: 
        d.append((Cap_current_shelter[i] - y[i])*daily_empty_spot_cost[0]*365)
    elif Cap_current_shelter[i] <= 100: 
        e.append((Cap_current_shelter[i] - y[i])*daily_empty_spot_cost[1]*365)
    else: 
        f.append((Cap_current_shelter[i] - y[i])*daily_empty_spot_cost[2]*365)

#total cost measured by: individual operating cost for current shelters per day * 365 day * number of people + operating cost per empty spot * 365 day * number of empty spot + construction fee for new shelters + individual operating cost for new shelters per day * 365 day * number of people       
#for cost <= government budget (in a year)       
model2.addConstr((sum(a)+sum(b)+sum(c)+sum(d)+sum(e)+sum(f)+sum(x[j]*construction_cost[j] + x[j]*daily_individual_cost[j]*365 for j in range(len(size_of_shelter)))) <= 663200000)



<gurobi.Constr *Awaiting Model Update*>

### Objective function setting

In [12]:
cost2 = sum(a)+sum(b)+sum(c) +sum(d)+sum(e)+sum(f)+ sum(x[j]*construction_cost[j] + x[j]*daily_individual_cost[j]*365 for j in range(len(size_of_shelter)))
model2.setObjective(cost2, GRB.MINIMIZE)
model2.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 77 rows, 78 columns and 231 nonzeros
Model fingerprint: 0x9ec661fe
Variable types: 0 continuous, 78 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+06]
  Objective range  [7e+03, 5e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 6e+08]
Found heuristic solution: objective 2.536700e+08
Presolve removed 75 rows and 72 columns
Presolve time: 0.00s
Presolved: 2 rows, 6 columns, 12 nonzeros
Variable types: 0 continuous, 6 integer (0 binary)

Root relaxation: objective 1.921533e+08, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.9215e+08    0    1 2.5367e+08 1.9215e+08  24.3%     -    0s
H    0     0                    1.933560e+08 1.9215e+08  0.62%     - 

In [13]:
for var in model2.getVars():
    if var.x >= 0:
        print(var.varName, "=", round(var.x, 0))

number_of_small = 1.0
number_of_medium = 1.0
number_of_large = 6.0
occupancy_s1 = 210.0
occupancy_s2 = 152.0
occupancy_s3 = 82.0
occupancy_s4 = 42.0
occupancy_s5 = 76.0
occupancy_s6 = 56.0
occupancy_s7 = 560.0
occupancy_s8 = 888.0
occupancy_s9 = 94.0
occupancy_s10 = 28.0
occupancy_s11 = 40.0
occupancy_s12 = 90.0
occupancy_s13 = 88.0
occupancy_s14 = 51.0
occupancy_s15 = 28.0
occupancy_s16 = 120.0
occupancy_s17 = 50.0
occupancy_s18 = 40.0
occupancy_s19 = 32.0
occupancy_s20 = 252.0
occupancy_s21 = 48.0
occupancy_s22 = 110.0
occupancy_s23 = 50.0
occupancy_s24 = 25.0
occupancy_s25 = 110.0
occupancy_s26 = 25.0
occupancy_s27 = 110.0
occupancy_s28 = 34.0
occupancy_s29 = 21.0
occupancy_s30 = 74.0
occupancy_s31 = 48.0
occupancy_s32 = 45.0
occupancy_s33 = 40.0
occupancy_s34 = 10.0
occupancy_s35 = 60.0
occupancy_s36 = 5.0
occupancy_s37 = 28.0
occupancy_s38 = 26.0
occupancy_s39 = 37.0
occupancy_s40 = 4.0
occupancy_s41 = 70.0
occupancy_s42 = 25.0
occupancy_s43 = 90.0
occupancy_s44 = 30.0
occupancy_s

### Solution from model2:
In this case, Gurobi suggests building 1 small shelter, 1 medium
shelter, and 6 large shelters to provide enough accommodation. The minimized annual cost is
$192,710,250, which is significantly larger than the optimal cost in the previous model, because
of the additional costs included.

Improving the model allows us to get a deeper understanding of the shelter services problem. As
we do research on the costs needed to operate a shelter, we realize that the actual cost can be
much more complicated than our calculations because of the government regulations and
administration. Formulating a good optimization model of such a real-world problem can be very
challenging, but it is worth doing since it benefits the government, the taxpayers, and most
importantly, the homeless people.

## Conclusion:

Based on our analysis, we would recommend the city of Toronto to build 1 shelter at the capacity
of 50 (small size ) , 1 shelter at the capacity of 100 (medium size) , and 6 shelters at the capacity
of 200 (large size) to provide enough accommodation. In addition, the city should close one of
the existing shelters (the shelter 71th with a capacity of 27) and keep the rest of the existing
shelters operating. This can improve the chance for the homelessness find shelters and maximize the capacity within the limited budget. In addition, small shelters can make sure, even though no
homelessness comes, the empty spot fee won’t be too high, and avoid the waste of resources.

## Further improvement

To make this project even more reasonable, we would consider these three points. 

First, we would try
to figure out how to allocate the position for the shelter location, by trying to analyze the spread
of homelessness across the city, particularly where they gather together, or where they just
randomly walk. After that, we want to solve this optimization allocation problem, maybe build a
large shelter in the downtown area which has a high density, and some small shelters across the
rest of Toronto. 

Second, the demand for the shelter changes with the season. There is more
demand for shelter in colder weather compared to warmer weather. It would be interesting to
re-evaluate the shelter allocation model semi-annually, which can provide a window of
opportunity for the government to reduce more cost without jeopardizing the quality of service.

Third, in the current pandemic most shelters are operating at reduced capacity, and this resulted
in some people stranded outside the shelter facility, this is not acceptable so the government
provides accommodation and cost associated with hotel accommodations should be considered
in the future implementation of the model.